## ATP NN

# Opening the Data

In [2]:
import json

N = 1500
date = '2018-08-27'
name = 'atp_python_' + date + '_1-' + str(N) + '.json'

with open(name) as d:
    players = [json.loads(line) for line in d]

# Height and Weight Study

In [5]:
study_NN = []

for i in range(0, len(players)):
    if 'height' in players[i].keys():
        if 'weight' in players[i].keys():
            if 'prize' in players[i].keys():
                rank = i + 1
                height = players[i]['height']
                weight = players[i]['weight']
                prize = players[i]['prize']
                
                # Taking out the weird zeros
                if height >= 50 and weight >= 50:
                    study_NN.append([rank, height, weight, prize])

### First Prototype

In [6]:
study_NN_shuffled = study_NN[:]
shuffle(study_NN_shuffled)

rank = [i for i,_,_,_ in study_NN_shuffled]

# Predictors
hw = [[i,j] for _,i,j,_ in study_NN_shuffled]
hw_a = array(hw)

# Target
cap = 10**6
prize_cap = [1 if k >= cap else 0 for _,_,_,k in study_NN_shuffled]
prize_a = array(prize_cap)
prize_a_bin = to_categorical(prize_a)

# The Model
model = Sequential()
model.add(Dense(20, activation = 'relu', input_shape = (2,)))
model.add(Dense(20, activation = 'relu'))
# model.add(Dense(10, activation = 'relu'))
model.add(Dense(2))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_training = model.fit(x = hw_a, y = prize_a_bin, epochs = 4, validation_split = 0.1)

Train on 954 samples, validate on 107 samples
Epoch 1/4
954/954 [==============================] - 0s 316us/step - loss: 13.5838 - acc: 0.1572 - val_loss: 13.4066 - val_acc: 0.1682
Epoch 2/4
954/954 [==============================] - 0s 36us/step - loss: 13.5838 - acc: 0.1572 - val_loss: 13.4066 - val_acc: 0.1682
Epoch 3/4
954/954 [==============================] - 0s 36us/step - loss: 13.5838 - acc: 0.1572 - val_loss: 13.4066 - val_acc: 0.1682
Epoch 4/4
954/954 [==============================] - 0s 36us/step - loss: 13.5838 - acc: 0.1572 - val_loss: 13.4066 - val_acc: 0.1682


## Final Version

With more proper train and test sets

In [7]:
from random import shuffle
from numpy import array
from keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential
from random import choice

study_NN_shuffled = study_NN[:]
shuffle(study_NN_shuffled)

rank = [i for i,_,_,_ in study_NN_shuffled]
prize = [i for _,_,_,i in study_NN_shuffled]

# Predictors
hw = [[i,j] for _,i,j,_ in study_NN_shuffled]
hw_a = array(hw)

# Target
cap = 10**6
prize_cap = [1 if k >= cap else 0 for _,_,_,k in study_NN_shuffled]
prize_a = array(prize_cap)
prize_a_bin = to_categorical(prize_a)

# Randomly Selecting 16 Millionaires

index_million = []
for i in range(0,len(prize_cap)):
    if prize_cap[i] == 1:
        index_million.append(i)

choices_M_list = []
for i in range(0, 16):
    choices = choice(index_million)
    choices_M_list.append(choices)
    index_million.remove(choices)
    
# Randomly Selecting 91 non millionaires

index_non_million = []
for i in range(0,len(prize_cap)):
    if prize_cap[i] == 0:
        index_non_million.append(i)
        
choices_NM_list = []
for i in range(0, 91):
    choices = choice(index_non_million)
    choices_NM_list.append(choices)
    index_non_million.remove(choices)
    
# Building the test and train sets

hw_train = hw[:]
hw_test = []
prize_cap_train = prize_cap[:]
prize_cap_test = []
for i in choices_M_list:
    hw_test.append(hw[i])
    hw_train.remove(hw[i])
    prize_cap_test.append(prize_cap[i])
    prize_cap_train.remove(prize_cap[i])
    
for i in choices_NM_list:
    hw_test.append(hw[i])
    hw_train.remove(hw[i])
    prize_cap_test.append(prize_cap[i])
    prize_cap_train.remove(prize_cap[i])
    
hw_train = array(hw_train)
hw_test = array(hw_test)
prize_cap_train = to_categorical(array(prize_cap_train))
prize_cap_test = to_categorical(array(prize_cap_test))

## The Model

In [ ]:
# The Model
model = Sequential()
model.add(Dense(30, activation = 'relu', input_shape = (2,)))
model.add(Dense(30, activation = 'relu'))
# model.add(Dense(10, activation = 'relu'))
model.add(Dense(2))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_training = model.fit(x = hw_train, y = prize_cap_train, epochs = 4, validation_data = (hw_test, prize_cap_test))